In [1]:
from dataset_UNSW_NB15 import Dataset_UNSW_NB15
from dataset_UNSW_NB15_1 import Dataset_UNSW_NB15_1
from dataset_NSL_KDD import Dataset_NSL_KDD
from dataset_IoT_NID import Dataset_IoT_NID
from experiment import Experiment
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)   


In [2]:
dataset = Dataset_UNSW_NB15()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run_noFS(k=500, c_attack=0.2, c_normal=0.001)
exp.run_pca(k=500, n_pca=30, c_attack=0.2, c_normal=0.001)
exp.run_fi(k=500, n_fi=80, c_attack=0.2, c_normal=0.001)
exp.run_hybrid(k=500, n_fi=20, n_pca=20, c_attack=0.2, c_normal=0.001)
exp.run_var(k=500, c_attack=0.2, c_normal=0.001)

X_ohe shape is: 161
X_num shape is: 38
------------------------------
Feature selection: noFS k:500
Accuracy: 0.73
Attack Subsystem Accuracy: 0.73
Normal Subsystem Accuracy: 0.52
F1 Score: 0.74
F1 Score Attack: 0.73
F1 Score Normal: 0.42
Fit Time: 22.74s
Evaluate Time for Total Data: 1.22s
------------------------------


In [ ]:
dataset = Dataset_NSL_KDD()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run_noFS(k=500, c_attack=0.1, c_normal=0.01)
exp.run_pca(k=500, n_pca=30, c_attack=0.1, c_normal=0.01)
exp.run_fi(k=500, n_fi=40, c_attack=0.1, c_normal=0.01)
exp.run_hybrid(k=500, n_fi=20, n_pca=20, c_attack=0.1, c_normal=0.01)
exp.run_var(k=500, c_attack=0.1, c_normal=0.01)

In [ ]:
dataset = Dataset_IoT_NID()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run_noFS(k=500)
exp.run_pca(k=500, n_pca=30)
exp.run_fi(k=500, n_fi=40)
exp.run_hybrid(k=500, n_fi=20, n_pca=20)
exp.run_var(k=500)

In [ ]:
dataset = Dataset_UNSW_NB15()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# c_attackとc_normalを0.0001から0.5まで10分割
c_values = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]

# 結果を格納するための配列を準備
results = np.zeros((len(c_values), len(c_values)))

# 各c_valueについて実験を行う
for i, c_attack in enumerate(c_values):
    for j, c_normal in enumerate(c_values):
        exp.run_noFS(k=500, c_attack=c_attack, c_normal=c_normal)
        results[i, j] = exp.accuracy

# ヒートマップを表示
plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(results), xticklabels=c_values, yticklabels=np.flipud(c_values), annot=True, vmax=1)
plt.xlabel('c_normal')
plt.ylabel('c_attack')
plt.title('Heatmap of experiment results')
plt.show()


In [ ]:
dataset = Dataset_NSL_KDD()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# c_attackとc_normalを0.0001から0.5まで10分割
c_values = np.linspace(0.01, 0.1, 5)

# 結果を格納するための配列を準備
results = np.zeros((len(c_values), len(c_values)))

# 各c_valueについて実験を行う
for i, c_attack in enumerate(c_values):
    for j, c_normal in enumerate(c_values):
        exp.run_noFS(k=500, c_attack=c_attack, c_normal=c_normal)
        results[i, j] = exp.accuracy

# ヒートマップを表示
plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(results), xticklabels=c_values, yticklabels=np.flipud(c_values), annot=True, vmax=1)
plt.xlabel('c_normal')
plt.ylabel('c_attack')
plt.title('Heatmap of experiment results')
plt.show()

In [ ]:
dataset = Dataset_IoT_NID()
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# c_attackとc_normalを0.0001から0.5まで10分割
c_values = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]

# 結果を格納するための配列を準備
results = np.zeros((len(c_values), len(c_values)))

# 各c_valueについて実験を行う
for i, c_attack in enumerate(c_values):
    for j, c_normal in enumerate(c_values):
        exp.run_noFS(k=500, c_attack=c_attack, c_normal=c_normal)
        results[i, j] = exp.accuracy

# ヒートマップを表示
plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(results), xticklabels=c_values, yticklabels=np.flipud(c_values), annot=True, vmax=1)
plt.xlabel('c_normal')
plt.ylabel('c_attack')
plt.title('Heatmap of experiment results')
plt.show()